<a href="https://colab.research.google.com/github/mfirmann/telco-ftth-performance-analysis/blob/main/data_generator_ftth.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import random
import numpy as np
from datetime import datetime, timedelta

# ==========================================
# 1. KONFIGURASI 100.000 DATA
# ==========================================

JUMLAH_DATA = 100000
START_DATE = datetime(2025, 1, 1) # Data setahun penuh

# --- LIST TEKNISI (20 Orang) ---
# Format: 'Nama': Skill_Factor
# < 1.0 = Cepat (Hero), > 1.0 = Lambat (Villain), 1.0 = Normal
teknisi_profile = {
    'Budi Santoso': 0.7,   'Andi Wijaya': 1.5,    'Citra Dewi': 0.8,
    'Dedi Kurniawan': 1.1, 'Eko Pratama': 1.0,    'Fajar Shodiq': 0.9,
    'Gilang Ramadhan': 1.2,'Haniifah': 0.8,       'Indra Gunawan': 1.0,
    'Joko Anwar': 1.3,     'Kurnia Mega': 0.9,    'Larasati': 1.0,
    'Maman Suherman': 1.1, 'Nina Zatulini': 0.8,  'Oki Setiana': 1.2,
    'Putri Titian': 1.0,   'Qori Sandioriva': 1.1,'Rudi Salim': 0.7,
    'Sinta Nuriyah': 1.0,  'Tono Sudirjo': 1.4
}
list_teknisi = list(teknisi_profile.keys())

# --- DAFTAR MASALAH (ROOT CAUSE) ---
# Format: 'Masalah': [Min_Jam, Max_Jam, Berat_Peluang]
root_cause_profile = {
    'Fiber Cut (Kabel Putus)': [4.0, 8.0, 5],    # Jarang, tapi lama banget
    'Redaman Tinggi (High Loss)': [1.0, 3.0, 25], # Sering
    'ONT/Modem Rusak': [0.5, 2.0, 20],           # Sering, cepat ganti
    'Konfigurasi Logic': [0.2, 1.0, 35],         # Paling sering, paling cepat
    'Gangguan Tiang/Kabel': [2.0, 5.0, 15]       # Lumayan sering
}
cause_keys = list(root_cause_profile.keys())
cause_weights = [item[2] for item in root_cause_profile.values()]

regions = ['BSD_Sektor_1', 'BSD_Sektor_2', 'BSD_Commercial', 'BSD_Residential', 'BSD_Edutown']
segments = ['Regular', 'Premium', 'VIP']

# ==========================================
# 2. GENERATOR ENGINE (OPTIMIZED)
# ==========================================
print(f"🔄 Sedang generate {JUMLAH_DATA} baris data... Mohon tunggu sebentar.")

data = []

for i in range(1, JUMLAH_DATA + 1):
    # --- A. Generate Atribut ---
    ticket_id = f"INC-{202500000+i}"
    technician = random.choice(list_teknisi)
    region = random.choice(regions)
    segment = random.choice(segments)

    # --- B. Generate Masalah ---
    cause = random.choices(cause_keys, weights=cause_weights, k=1)[0]
    base_min, base_max, _ = root_cause_profile[cause]

    # --- C. Generate Waktu (Incident) ---
    # Random hari dalam setahun (0-365)
    rand_day = random.randint(0, 365)
    # Pola jam kerja: Lebih banyak tiket di jam 08:00 - 16:00
    hour_pattern = random.choices(
        [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23],
        weights=[1,1,1,1,2,3,5,8,10,10,10,8,8,10,10,10,8,5,4,3,2,2,1,1], k=1
    )[0]

    incident_time = START_DATE + timedelta(days=rand_day, hours=hour_pattern, minutes=random.randint(0, 59))

    # --- D. Kalkulasi Durasi (Logic Inti) ---
    tech_factor = teknisi_profile[technician]
    random_chaos = random.uniform(0.8, 1.2) # Variasi alamiah

    # Durasi Final = (Random Base * Kecepatan Teknisi * Faktor Chaos)
    final_duration = random.uniform(base_min, base_max) * tech_factor * random_chaos

    # VIP Bonus (Lebih cepat 20%)
    if segment == 'VIP':
        final_duration *= 0.8

    restored_time = incident_time + timedelta(hours=final_duration)

    # --- E. Status SLA (Target 4 Jam) ---
    sla_status = 'Breach' if final_duration > 4.0 else 'Comply'

    data.append([
        ticket_id, technician, region, segment, cause,
        incident_time.strftime('%Y-%m-%d %H:%M:%S'),
        restored_time.strftime('%Y-%m-%d %H:%M:%S'),
        round(final_duration, 2),
        sla_status
    ])

# ==========================================
# 3. EXPORT TO CSV
# ==========================================
cols = ['Ticket_ID', 'Technician', 'Region', 'Customer_Type',
        'Root_Cause', 'Open_Time', 'Close_Time', 'Duration_Hours', 'SLA_Flag']

df = pd.DataFrame(data, columns=cols)

# Simpan
filename = 'ftth_bsd_100k_full.csv'
df.to_csv(filename, index=False)

print("\n" + "="*40)
print(f"✅ SUKSES! File '{filename}' berhasil dibuat.")
print(f"📊 Total Data: {len(df)} Baris")
print(f"💾 Ukuran File: Estimasi sekitar ~8-10 MB")
print("="*40)
print("\n⚠️ PENTING UNTUK LAPTOP KENTANG:")
print("1. JANGAN double-click file CSV ini untuk dibuka di Excel (Bisa Not Responding).")
print("2. Gunakan 'df.head()' di sini atau 'DB Browser for SQLite' untuk melihat isinya.")
print("3. File siap di-import ke Tableau/SQL.")

# Preview sedikit
print("\nContoh 5 Data Teratas:")
display(df.head())

🔄 Sedang generate 100000 baris data... Mohon tunggu sebentar.

✅ SUKSES! File 'ftth_bsd_100k_full.csv' berhasil dibuat.
📊 Total Data: 100000 Baris
💾 Ukuran File: Estimasi sekitar ~8-10 MB

⚠️ PENTING UNTUK LAPTOP KENTANG:
1. JANGAN double-click file CSV ini untuk dibuka di Excel (Bisa Not Responding).
2. Gunakan 'df.head()' di sini atau 'DB Browser for SQLite' untuk melihat isinya.
3. File siap di-import ke Tableau/SQL.

Contoh 5 Data Teratas:


,Ticket_ID,Technician,Region,Customer_Type,Root_Cause,Open_Time,Close_Time,Duration_Hours,SLA_Flag
0,INC-202500001,Putri Titian,BSD_Commercial,VIP,Konfigurasi Logic,2025-09-21 07:30:00,2025-09-21 08:04:09,0.57,Comply
1,INC-202500002,Qori Sandioriva,BSD_Sektor_1,VIP,Redaman Tinggi (High Loss),2025-08-11 16:11:00,2025-08-11 18:23:04,2.20,Comply
2,INC-202500003,Gilang Ramadhan,BSD_Sektor_1,Regular,ONT/Modem Rusak,2025-02-13 07:26:00,2025-02-13 09:36:11,2.17,Comply
3,INC-202500004,Citra Dewi,BSD_Edutown,VIP,Konfigurasi Logic,2025-12-25 15:36:00,2025-12-25 16:08:04,0.53,Comply
4,INC-202500005,Tono Sudirjo,BSD_Sektor_1,Premium,ONT/Modem Rusak,2025-04-04 16:38:00,2025-04-04 18:53:32,2.26,Comply
